In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.metrics import mean_squared_error 
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

# Ouline
- Outlier analysis
- Forecasting horizon explained
- Time series cross validation
- Lag Features
- Predicting the Future

In [4]:
df = pd.read_csv('../input/hourly-energy-consumption/PJME_hourly.csv')
df = df.set_index('Datetime')
df.index = pd.to_datetime(df.index)

In [5]:
df.head()

In [8]:
df.plot(style='.',
       figsize=(15,5),
       color=color_pal[0],
       title='PJME Energy Use in MW')
plt.show()

# 1. Outlier Analysis and removal

In [9]:
df['PJME_MW'].plot(kind='hist', bins=500)

In [11]:
df.query('PJME_MW < 20_000')

In [13]:
df.query('PJME_MW <19_000').plot(figsize=(15,5),style='.')

In [14]:
df = df.query('PJME_MW > 19_000').copy()

# Time Series Cross Validation

In [15]:
from sklearn.model_selection import TimeSeriesSplit

In [16]:
tss = TimeSeriesSplit(n_splits=5,
                     test_size=24*365*1,
                     gap=24)
df = df.sort_index()


In [22]:
fig, axs = plt.subplots(5,1,figsize=(15,15),
                            sharex=True)
fold = 0
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]
    train['PJME_MW'].plot(ax=axs[fold],
                         label='Training Set',
                         title=f'Data Train/Test Split Fold {fold}')
    test['PJME_MW'].plot(ax=axs[fold], 
                         label='Test Set')
    axs[fold].axvline(test.index.min(),color='black',ls='--')
    fold += 1
plt.show()

# Forecasting Horizon Explained

In [24]:
def create_features(df):
    df = df.copy()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week
    return df
df = create_features(df)

# 3. Lag Features
- What was the target(x) days in the past

In [32]:
def add_lags(df):
    target_map = df['PJME_MW'].to_dict()
    df['lag1'] = (df.index - pd.Timedelta('364 days')).map(target_map)
    df['lag2'] = (df.index - pd.Timedelta('728 days')).map(target_map)
    df['lag3'] = (df.index - pd.Timedelta('1092 days')).map(target_map)
    return df

In [33]:
df = add_lags(df)

## Train Using Cross Validation

In [38]:
tss = TimeSeriesSplit(n_splits=5,test_size=24*365*1,gap=24)
df = df.sort_index()

fold = 0
preds = []
scores = []
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]
    
    train = create_features(train)
    test = create_features(test)
    
    FEATURES = [
        'dayofyear', 'hour', 'dayofweek', 'quarter', 'month','year',
        'lag1','lag2','lag3'
    ]
    TARGET = 'PJME_MW'
    
    X_train = train[FEATURES]
    y_train = train[TARGET]
    
    X_test = test[FEATURES]
    y_test = test[TARGET]
    
    reg = xgb.XGBRegressor(base_score=0.5,booster='gbtree',
                          n_estimators=1000,
                          early_stopping_rounds=50,
                          objective='reg:linear',
                          max_depth=3,
                          learning_rate=0.01)
    reg.fit(X_train,y_train,
           eval_set=[(X_train,y_train), (X_test,y_test)],
           verbose=100)
    y_pred = reg.predict(X_test)
    preds.append(y_pred)
    score = np.sqrt(mean_squared_error(y_test,y_pred))
    scores.append(score)
    

In [39]:
print(f'Score across folds {np.mean(scores):0.4f}')
print(f'Fold scores:{scores}')

# Predicting the Future
- Retraining on all data
- To predict the future we need an empty dataframe for future data ranges
- Run those dates through our feature creation code + lag creation

In [40]:
# Retrain on all data
df = create_features(df)

FEATURES = [
    'dayofyear', 'hour', 'dayofweek', 'quarter', 'month', 'year',
    'lag1','lag2','lag3'
]
TARGET = 'PJME_MW'
X_all = df[FEATURES]
y_all = df[TARGET]

reg = xgb.XGBRegressor(base_score=0.5,
                      booster='gbtree',
                      n_estimators=500,
                      objective='reg:linear',
                      max_depth=3,
                      learning_rate=0.01)
reg.fit(X_all,y_all,
       eval_set=[(X_all,y_all)],
       verbose=100)

In [41]:
df.index.max()

In [44]:
# Create future dataframe
future = pd.date_range('2018-08-03', '2019-08-01',freq='1h')
future_df = pd.DataFrame(index=future)
future_df['isFuture'] = True
df['isFuture'] = False
df_and_futrue = pd.concat([df, future_df])

In [46]:
df_and_futrue = create_features(df_and_futrue)
df_and_futrue = add_lags(df_and_futrue)

In [47]:
future_w_features = df_and_futrue.query('isFuture').copy()

In [48]:
future_w_features

## Predict the future

In [50]:
future_w_features['pred'] = reg.predict(future_w_features[FEATURES])

In [51]:
future_w_features['pred'].plot(figsize=(10,5),
                              color=color_pal[4],
                              ms=1,lw=1,title='Future Predictions')

# Bonus: Saving Model For later

In [53]:
reg.save_model('model.json')

In [55]:
!ls -lh

In [57]:
reg_new = xgb.XGBRegressor()
reg_new.load_model('model.json')